In [20]:
import json

file_path = "geometry.json"

with open(file_path, 'r') as file:
    data = json.load(file)
    
print(data)
features = data["features"]

{'type': 'geoFencingGeometry', 'features': [{'properties': {'id': 'box1', 'name': 'Zone A'}, 'geometry': [[-122.42, 37.78], [-122.41, 37.78], [-122.41, 37.77], [-122.42, 37.77], [-122.42, 37.78]]}, {'properties': {'id': 'box1', 'name': 'Extra Zone A'}, 'geometry': [[-122.42, 37.78], [-122.41, 37.78], [-122.41, 37.77], [-122.42, 37.77], [-122.42, 37.78]]}, {'properties': {'id': 'bigbox', 'name': 'BIG ZONE'}, 'geometry': [[-125.0, 49.3843], [-66.9346, 49.3843], [-66.9346, 24.3963], [-125.0, 24.3963], [-125.0, 49.3843]]}, {'properties': {'id': 'box2', 'name': 'Zone B'}, 'geometry': [[-122.4, 37.79], [-122.39, 37.79], [-122.39, 37.78], [-122.4, 37.78], [-122.4, 37.79]]}, {'properties': {'id': 'box3', 'name': 'Zone C'}, 'geometry': [[-122.43, 37.76], [-122.42, 37.76], [-122.42, 37.75], [-122.43, 37.75], [-122.43, 37.76]]}, {'properties': {'id': 'box4', 'name': 'Zone D'}, 'geometry': [[-122.38, 37.8], [-122.37, 37.8], [-122.37, 37.79], [-122.38, 37.79], [-122.38, 37.8]]}, {'properties': {'id

In [21]:
def is_point_inside_polygon(x, y, polygon):
    """
    Determines if a point (x, y) is inside a given polygon using the Ray-Casting method.
    """
    n = len(polygon)
    inside = False

    p1x, p1y = polygon[0]
    for i in range(n + 1):
        p2x, p2y = polygon[i % n]
        if min(p1y, p2y) < y <= max(p1y, p2y) and x <= max(p1x, p2x):
            if p1y != p2y:
                xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
            if p1x == p2x or x <= xinters:
                inside = not inside
        p1x, p1y = p2x, p2y

    return inside

In [22]:
point_x, point_y = -122.415, 37.775
is_inside = is_point_inside_polygon(point_x, point_y, features[0]["geometry"])

print(f"Is the point inside Zone A? {is_inside}")

Is the point inside Zone A? True


In [23]:
def getFences(x, y, geometry):
    box_list = []
    for i in geometry:
        if is_point_inside_polygon(x, y, i["geometry"]):
            box_list.append(i["properties"]["name"])
            
    return box_list

In [24]:
import pandas as pd


drivefile = "0_130000_ciaran_donuts.csv"

df = pd.read_csv(drivefile)

In [ ]:
import pandas as pd
def divideDataFrame(df, geometry):
    """
    Divides a DataFrame into multiple DataFrames based on the zones defined in the geometry.
    Each zone will contain the rows that fall within its boundaries.
    """
        
    zone_dict = {} 
    final_zone_dict = {}
    zone_last_seen = {}
    index_counter = 0

    if "Latitude|Degrees|-180.0|180.0|25" not in df.columns or "Longitude|Degrees|-180.0|180.0|25" not in df.columns:
        raise ValueError("Latitude or Longitude column missing in DataFrame.")

    # Process each row in the DataFrame
    for _, row in df.iterrows():
        y = row["Latitude|Degrees|-180.0|180.0|25"]
        x = row["Longitude|Degrees|-180.0|180.0|25"]

        if pd.isna(x) or pd.isna(y):
            continue 

        areas = getFences(x, y, features)

        for area in areas:
            if area in zone_last_seen:
            
                if index_counter - zone_last_seen[area] > 2:
                    # Save the current data to final_zone_dict before resetting
                    if area not in final_zone_dict:
                        final_zone_dict[area] = []

                    final_zone_dict[area].append(pd.DataFrame(zone_dict[area]))  # Store previous data segment
                    zone_dict[area] = []

            # Update last seen valid index
            zone_last_seen[area] = index_counter

            # Add row to zone_dict
            if area not in zone_dict:
                zone_dict[area] = []
            zone_dict[area].append(row.to_dict())

        if areas:
            index_counter += 1 

    # At the end, save remaining data in zone_dict to final_zone_dict
    for zone, rows in zone_dict.items():
        if zone not in final_zone_dict:
            final_zone_dict[zone] = []
        final_zone_dict[zone].append(pd.DataFrame(rows))
        
    return final_zone_dict

In [ ]:


zone_dict = {} 
final_zone_dict = {}
zone_last_seen = {}
index_counter = 0

if "Latitude|Degrees|-180.0|180.0|25" not in df.columns or "Longitude|Degrees|-180.0|180.0|25" not in df.columns:
    raise ValueError("Latitude or Longitude column missing in DataFrame.")

# Process each row in the DataFrame
for _, row in df.iterrows():
    y = row["Latitude|Degrees|-180.0|180.0|25"]
    x = row["Longitude|Degrees|-180.0|180.0|25"]

    if pd.isna(x) or pd.isna(y):
        continue 

    areas = getFences(x, y, features)

    for area in areas:
        if area in zone_last_seen:
           
            if index_counter - zone_last_seen[area] > 2:
                # Save the current data to final_zone_dict before resetting
                if area not in final_zone_dict:
                    final_zone_dict[area] = []

                final_zone_dict[area].append(pd.DataFrame(zone_dict[area]))  # Store previous data segment
                zone_dict[area] = []

        # Update last seen valid index
        zone_last_seen[area] = index_counter

        # Add row to zone_dict
        if area not in zone_dict:
            zone_dict[area] = []
        zone_dict[area].append(row.to_dict())

    if areas:
        index_counter += 1 

# At the end, save remaining data in zone_dict to final_zone_dict
for zone, rows in zone_dict.items():
    if zone not in final_zone_dict:
        final_zone_dict[zone] = []
    final_zone_dict[zone].append(pd.DataFrame(rows))

# Print the CSV-like output for each segment
for zone, df_list in final_zone_dict.items():
    print(f"Zone: {zone}")
    for i, df in enumerate(df_list):
        print(f"Segment {i+1}:")
        print(df)
        print("\n")


Zone: splitzone
Segment 1:
     Unnamed: 0  Interval|"ms"|0|0|1  Utc|"ms"|0|0|1  \
0        2853.0               5923.0    1.714855e+12   
1        2873.0               5963.0    1.714855e+12   
2        2893.0               6003.0    1.714855e+12   
3        2913.0               6043.0    1.714855e+12   
4        2933.0               6083.0    1.714855e+12   
..          ...                  ...             ...   
103      4913.0              10043.0    1.714855e+12   
104      4933.0              10083.0    1.714855e+12   
105      4953.0              10123.0    1.714855e+12   
106      4973.0              10163.0    1.714855e+12   
107      4993.0              10203.0    1.714855e+12   

     Analog4|"Volts"|0.0|5.0|500  Analog5|"Volts"|0.0|5.0|500  \
0                           1.50                         1.93   
1                           1.46                         1.89   
2                           1.52                         1.94   
3                           1.49        